In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField
from pyspark.sql.functions import regexp_extract, lit, split, col

In [3]:
from datetime import datetime
import requests
import json
import os
import configparser
from pathlib import Path

In [ ]:
spark = SparkSession \
        .builder \
        .master('local') \
        .appName('tempera_obs') \
        .config("spark.driver.extraClassPath", "/Users/mdgome/Downloads/mysql-connector-java-8.0.28/mysql-connector-java-8.0.28.jar") \
        .getOrCreate()
#        .master('spark://hadoop01:7077') \
       # .config("spark.driver.extraClassPath", "/hadoop/jupyter_dir/jar/mysql-connector-java-8.0.28/mysql-connector-java-8.0.28.jar") \
            # .config("spark.driver.extraClassPath", "hdfs:///spark3-jars/mysql-connector-java-8.0.28.jar") \

In [5]:
now_datetime=datetime.now().strftime('%Y.%m.%d')
now_datetime # 디버깅 용

'2022.03.09'

In [6]:
config = configparser.ConfigParser()
config.read(Path("./config/config.ini"),encoding='utf-8')

['config/config.ini']

In [7]:
service_key=config['api']['service_key']
url="http://www.khoa.go.kr/api/oceangrid/ObsServiceObj/search.do?ServiceKey="+service_key+"&ResultType=json"
url # 디버깅용

'http://www.khoa.go.kr/api/oceangrid/ObsServiceObj/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ResultType=json'

In [ ]:
response = requests.get(url).json()
# 디버깅 용
# response['result']['data']

In [10]:
df = spark.createDataFrame(response["result"]["data"])
df.show(5)

+----------+---------+----------+----------------------------------+-----------+-------------+
| data_type|  obs_lat|   obs_lon|                        obs_object|obs_post_id|obs_post_name|
+----------+---------+----------+----------------------------------+-----------+-------------+
|조위관측소|35.024178|128.810933|조위,수온,염분,기온,기압,풍속,풍향|    DT_0063|       가덕도|
|조위관측소|37.731944|126.522222|                         조위,기압|    DT_0032|     강화대교|
|조위관측소|34.028333|127.308889|조위,수온,염분,기온,기압,풍속,풍향|    DT_0031|       거문도|
|조위관측소|34.801389|128.699167|조위,수온,염분,기온,기압,풍속,풍향|    DT_0029|       거제도|
|조위관측소|37.560833|126.601111|          조위,기온,기압,풍속,풍향|    DT_0058|       경인항|
+----------+---------+----------+----------------------------------+-----------+-------------+
only showing top 5 rows



In [11]:
config = configparser.ConfigParser()
config.read(Path("./config/config.ini"),encoding='utf-8')
# config.read(os.getcwd()+os.sep+'config'+os.sep+'config.ini',encoding='utf-8')

user = config['dev_mysql']['user']
password = config['dev_mysql']['password']
host = config['dev_mysql']['host']
port = config['dev_mysql']['port']
dbname = config['dev_mysql']['dbname']
url = config['dev_mysql']['url'].format(host=host,port=port,dbname=dbname)
# dbtable = config['dev_mysql']['dbtable']

In [13]:
df.write.format('jdbc').options(
    url=url,
    driver='com.mysql.jdbc.Driver',
    dbtable='obs_info',
    user=user,
    password=password).mode('overwrite').save()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


In [21]:
!hdfs dfs -ls /middle_project

Found 2 items
drwxr-xr-x   - hadoop supergroup          0 2022-02-26 05:47 /middle_project/2022.02.26_obs_info.csv
drwxr-xr-x   - hadoop supergroup          0 2022-03-02 17:35 /middle_project/2022.03.02_obs_info.csv


In [19]:
file_name = "hdfs:///middle_project/"+now_datetime+"_obs_info.csv"
file_name

'hdfs:///middle_project/2022.03.02_obs_info.csv'

In [20]:
# df.write.csv(file_name,header=True)
df.coalesce(1).write.format('com.databricks.spark.csv').save(file_name,header = 'true')

In [38]:
test_df = spark.read.csv(file_name)

In [39]:
test_df.show(5)

+----------+---------+----------+----------------------------------+-----------+-------------+
|       _c0|      _c1|       _c2|                               _c3|        _c4|          _c5|
+----------+---------+----------+----------------------------------+-----------+-------------+
| data_type|  obs_lat|   obs_lon|                        obs_object|obs_post_id|obs_post_name|
|조위관측소|35.024178|128.810933|조위,수온,염분,기온,기압,풍속,풍향|    DT_0063|       가덕도|
|조위관측소|37.731944|126.522222|                         조위,기압|    DT_0032|     강화대교|
|조위관측소|34.028333|127.308889|조위,수온,염분,기온,기압,풍속,풍향|    DT_0031|       거문도|
|조위관측소|34.801389|128.699167|조위,수온,염분,기온,기압,풍속,풍향|    DT_0029|       거제도|
+----------+---------+----------+----------------------------------+-----------+-------------+
only showing top 5 rows



In [19]:
spark.stop()